# side model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import SIDE

df_features = pd.read_parquet("data/features.parquet")

features = df_features[SIDE]
label = np.load("data/side_label.npy")
len_gap = len(label) - len(df_features)
label = label[len_gap:]

print(features.shape)
print(label.shape)

mask = features.index < date_to_timestamp("2025-01-01")
features_masked = features[mask]
label_masked = label[mask]

print(features_masked.shape)
print(label_masked.shape)
print(np.unique(label_masked, return_counts=True))

features.isna().sum().sort_values(ascending=False)

(15677, 495)
(15677,)
(14637, 495)
(14637,)
(array([0, 1]), array([6738, 7899]))


williams_r                         0
stc_dt_lag5                        0
td_sequential_buy                  0
evenbetter_sinewave_short_lag12    0
adaptive_bp_lead_dt                0
                                  ..
bandpass_lag5                      0
aroon_diff_dt                      0
stc_ddt                            0
adaptive_stochastic_ddt_lag1       0
fti_dt_lag19                       0
Length: 495, dtype: int64

In [2]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(features_masked, label_masked)
res = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True)
res.keys()

dict_keys(['valid auc-mean', 'valid auc-stdv'])

In [3]:
res["valid auc-mean"][-1]

0.917259281092343

In [4]:
import lightgbm as lgb
import optuna


def objective(trial):
    METRIC = "auc"

    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(features, label)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        metrics=METRIC,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-06 00:50:55,474] A new study created in memory with name: no-name-a7c2c1f0-068e-4620-a421-3abb64feadc3
[I 2025-06-06 00:50:58,862] Trial 0 finished with value: 0.9114358218493985 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 236, 'max_depth': 926, 'min_gain_to_split': 0.750460659918505, 'min_data_in_leaf': 318, 'lambda_l1': 7.6736268703256725, 'lambda_l2': 9.878803163568897, 'num_boost_round': 103}. Best is trial 0 with value: 0.9114358218493985.
[I 2025-06-06 00:51:01,096] Trial 1 finished with value: 0.8993296450029185 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 91, 'max_depth': 780, 'min_gain_to_split': 0.23954190978928425, 'min_data_in_leaf': 421, 'lambda_l1': 90.13589015146184, 'lambda_l2': 59.41787920030896, 'num_boost_round': 299}. Best is trial 0 with value: 0.9114358218493985.
[I 2025-06-06 00:51:17,027] Trial 2 finished with value: 0.9019194814892133 and parameters: 

In [6]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 140,
 'max_depth': 535,
 'min_gain_to_split': 0.06826052345821144,
 'min_data_in_leaf': 178,
 'lambda_l1': 1.5771905785425173,
 'lambda_l2': 15.901262859253215,
 'num_boost_round': 762}

In [7]:
import lightgbm as lgb  # noqa

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(features_masked, label_masked)
side_model = lgb.train(params, dtrain)
side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [10]:
import plotly.express as px

test_features = features[features.index > date_to_timestamp("2025-01-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [11]:
side_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_side.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcEntropyBarV1/model/model_side_prod.txt"
)

# meta model

In [2]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(15677, 877)
(15677,)
(14637, 877)
(14637,)


williams_r                       0
price_variance_ratio_lag1        0
acp_pwr_16                       0
entropy_for_jesse_dt_lag11       0
norm_on_balance_volume_lag1      0
                                ..
homodyne                         0
acp_pwr_26                       0
mod_stochastic_ddt_lag1          0
bekker_parkinson_vol_dt_lag16    0
model                            0
Length: 877, dtype: int64

In [3]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="macro")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [4]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-06 16:28:24,195] A new study created in memory with name: no-name-ea289057-530f-44e2-9805-074227788d46
[I 2025-06-06 16:28:44,191] Trial 0 finished with value: 0.7600236769955208 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 437, 'max_depth': 849, 'min_gain_to_split': 0.21648108195504231, 'min_data_in_leaf': 110, 'lambda_l1': 27.515551728009715, 'lambda_l2': 85.10200257109858, 'num_boost_round': 567}. Best is trial 0 with value: 0.7600236769955208.
[I 2025-06-06 16:28:59,385] Trial 1 finished with value: 0.7209312180529206 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 58, 'max_depth': 390, 'min_gain_to_split': 0.5396495248522933, 'min_data_in_leaf': 204, 'lambda_l1': 60.854721180161725, 'lambda_l2': 5.576907525090653, 'num_boost_round': 549}. Best is trial 0 with value: 0.7600236769955208.
[I 2025-06-06 16:29:09,028] Trial 2 finished with value: 0.8074741673052435 and parameters:

In [5]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 304,
 'max_depth': 440,
 'min_gain_to_split': 0.5327424723827091,
 'min_data_in_leaf': 168,
 'lambda_l1': 0.42422798189659605,
 'lambda_l2': 2.5085221847412402,
 'num_boost_round': 719}

In [6]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")